<a href="https://colab.research.google.com/github/hemasrikamana18/Final-projects/blob/main/Medical_report_simplifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!apt-get update -qq
!apt-get install -y poppler-utils tesseract-ocr
!pip install pyngrok
!pip install -q openai pdfplumber pytesseract pillow pdf2image flask

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 77 not upgraded.


In [ ]:
%%writefile app.py
import os
os.environ["OPENROUTER_API_KEY"] = "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"

from flask import Flask, render_template, request, redirect, url_for, session, flash
from werkzeug.utils import secure_filename
from werkzeug.security import generate_password_hash, check_password_hash
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
from openai import OpenAI
import json
import re
import functools

app = Flask(__name__)
app.secret_key = "medical_simplifier_secret_2024"

# 🔐 OpenRouter API key
api_key = "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"

client = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)

UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# ─── Simple in-memory user store (swap with DB in production) ───
USERS = {}   # { username: hashed_password }

# ─── Login required decorator ───
def login_required(f):
    @functools.wraps(f)
    def decorated(*args, **kwargs):
        if "user" not in session:
            return redirect(url_for("login"))
        return f(*args, **kwargs)
    return decorated


# ─── Text extraction (poppler-safe) ───
def extract_text(file_path):
    text = ""

    if file_path.lower().endswith(".pdf"):
        # Try pdfplumber first (no poppler needed)
        try:
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
        except Exception:
            pass

        # Only use pdf2image (poppler) if pdfplumber found nothing
        if not text.strip():
            try:
                images = convert_from_path(file_path)
                for img in images:
                    text += pytesseract.image_to_string(img) + "\n"
            except Exception as e:
                raise ValueError(
                    f"Could not extract text from PDF. "
                    f"If this is a scanned PDF, make sure poppler-utils is installed. "
                    f"(Detail: {e})"
                )
        return text

    elif file_path.lower().endswith((".png", ".jpg", ".jpeg")):
        img = Image.open(file_path)
        return pytesseract.image_to_string(img)

    elif file_path.lower().endswith(".txt"):
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()

    else:
        raise ValueError("Unsupported file format")


def parse_ai_response(text):
    """Try to extract structured sections from the AI response."""
    sections = {
        "abnormal_findings": "",
        "simple_explanation": "",
        "possible_conditions": "",
        "severity": "",
        "next_steps": "",
        "disclaimer": ""
    }

    patterns = {
        "abnormal_findings": r"(?:1\.|abnormal findings?)[:\s]*(.*?)(?=(?:2\.|explain|simple|possible|predict|severity|next|disclaimer)|$)",
        "simple_explanation": r"(?:2\.|explain|simple explanation)[:\s]*(.*?)(?=(?:3\.|possible|predict|severity|next|disclaimer)|$)",
        "possible_conditions": r"(?:3\.|possible (?:diseases?|conditions?))[:\s]*(.*?)(?=(?:4\.|severity|next|disclaimer)|$)",
        "severity": r"(?:4\.|severity)[:\s]*(.*?)(?=(?:5\.|next steps?|disclaimer)|$)",
        "next_steps": r"(?:5\.|next steps?|suggest)[:\s]*(.*?)(?=(?:6\.|disclaimer)|$)",
        "disclaimer": r"(?:6\.|disclaimer)[:\s]*(.*?)$"
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
        if match:
            sections[key] = match.group(1).strip()

    if not any(sections.values()):
        sections["simple_explanation"] = text

    return sections


# ─── Auth Routes ───

@app.route("/login", methods=["GET", "POST"])
def login():
    if "user" in session:
        return redirect(url_for("home"))
    error = ""
    if request.method == "POST":
        username = request.form.get("username", "").strip()
        password = request.form.get("password", "")
        user_hash = USERS.get(username)
        if user_hash and check_password_hash(user_hash, password):
            session["user"] = username
            return redirect(url_for("home"))
        else:
            error = "Invalid username or password."
    return render_template("login.html", error=error)


@app.route("/register", methods=["GET", "POST"])
def register():
    if "user" in session:
        return redirect(url_for("home"))
    error = ""
    if request.method == "POST":
        username = request.form.get("username", "").strip()
        password = request.form.get("password", "")
        confirm  = request.form.get("confirm", "")
        if not username or not password:
            error = "Username and password are required."
        elif password != confirm:
            error = "Passwords do not match."
        elif username in USERS:
            error = "Username already exists. Please choose another."
        else:
            USERS[username] = generate_password_hash(password)
            session["user"] = username
            return redirect(url_for("home"))
    return render_template("register.html", error=error)


@app.route("/logout")
def logout():
    session.clear()
    return redirect(url_for("login"))


# ─── Main Routes (protected) ───

@app.route("/")
@login_required
def home():
    return render_template("home.html", user=session.get("user"))


@app.route("/analyze", methods=["GET", "POST"])
@login_required
def analyze():
    error = ""

    if request.method == "POST":
        file = request.files.get("report")

        if not file or file.filename == "":
            error = "Please upload a medical report file."
        else:
            try:
                filename = secure_filename(file.filename)
                file_path = os.path.join(UPLOAD_FOLDER, filename)
                file.save(file_path)

                report_text = extract_text(file_path)

                if not report_text.strip():
                    raise ValueError("Could not extract text from file. It may be scanned or corrupted.")

                report_text = report_text[:8000]

                prompt = f"""
You are a medical assistant helping patients understand their medical reports in simple language.

Analyze the following medical report and respond in this EXACT structured format:

1. Abnormal Findings:
[List all abnormal values or findings clearly]

2. Simple Explanation:
[Explain each finding in plain, easy-to-understand language without medical jargon]

3. Possible Conditions:
[List possible conditions these findings may indicate — NOT a diagnosis]

4. Severity:
[Indicate if findings are Mild / Moderate / Severe and why]

5. Next Steps:
[Recommend what the patient should do next — see a doctor, tests, lifestyle changes, etc.]

6. Disclaimer:
[Standard medical disclaimer reminding this is not a diagnosis]

Medical Report:
{report_text}
"""

                response = client.chat.completions.create(
                    model="openai/gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": "You are a helpful, empathetic medical report interpreter who explains complex medical terms in simple, clear language."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.2
                )

                raw_result = response.choices[0].message.content
                parsed = parse_ai_response(raw_result)
                parsed["raw"] = raw_result
                parsed["filename"] = filename

                session["result"] = parsed
                return redirect(url_for("result"))

            except Exception as e:
                error = f"Error processing file: {str(e)}"

    return render_template("analyze.html", error=error, user=session.get("user"))


@app.route("/result")
@login_required
def result():
    data = session.get("result")
    if not data:
        return redirect(url_for("analyze"))
    return render_template("result.html", data=data, user=session.get("user"))


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)

Overwriting app.py


In [ ]:
!mkdir -p templates static uploads

In [ ]:
%%writefile templates/login.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Login — MediScan AI</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="auth-page">
    <div class="bg-particles">
        <div class="particle"></div><div class="particle"></div>
        <div class="particle"></div><div class="particle"></div>
    </div>

    <div class="auth-wrapper">
        <div class="auth-card animate-slide-up">
            <div class="auth-logo">
                <span>🏥</span>
                <span class="nav-title">MediScan <span class="highlight">AI</span></span>
            </div>
            <h2 class="auth-title">Welcome back</h2>
            <p class="auth-subtitle">Sign in to analyze your medical reports</p>

            {% if error %}
            <div class="error-alert">
                <span>⚠️</span><span>{{ error }}</span>
            </div>
            {% endif %}

            <form method="post" class="auth-form">
                <div class="form-group">
                    <label>Username</label>
                    <input type="text" name="username" placeholder="Enter your username" required autocomplete="username">
                </div>
                <div class="form-group">
                    <label>Password</label>
                    <input type="password" name="password" placeholder="Enter your password" required autocomplete="current-password">
                </div>
                <button type="submit" class="btn-auth">
                    <span>🔐 Sign In</span>
                </button>
            </form>

            <p class="auth-switch">
                Don't have an account? <a href="/register">Create one</a>
            </p>
        </div>
    </div>
</body>
</html>

Overwriting templates/login.html


In [ ]:
%%writefile templates/register.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Register — MediScan AI</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="auth-page">
    <div class="bg-particles">
        <div class="particle"></div><div class="particle"></div>
        <div class="particle"></div><div class="particle"></div>
    </div>

    <div class="auth-wrapper">
        <div class="auth-card animate-slide-up">
            <div class="auth-logo">
                <span>🏥</span>
                <span class="nav-title">MediScan <span class="highlight">AI</span></span>
            </div>
            <h2 class="auth-title">Create account</h2>
            <p class="auth-subtitle">Join MediScan AI to start analyzing reports</p>

            {% if error %}
            <div class="error-alert">
                <span>⚠️</span><span>{{ error }}</span>
            </div>
            {% endif %}

            <form method="post" class="auth-form">
                <div class="form-group">
                    <label>Username</label>
                    <input type="text" name="username" placeholder="Choose a username" required autocomplete="username">
                </div>
                <div class="form-group">
                    <label>Password</label>
                    <input type="password" name="password" placeholder="Create a password" required autocomplete="new-password">
                </div>
                <div class="form-group">
                    <label>Confirm Password</label>
                    <input type="password" name="confirm" placeholder="Confirm your password" required autocomplete="new-password">
                </div>
                <button type="submit" class="btn-auth">
                    <span>✨ Create Account</span>
                </button>
            </form>

            <p class="auth-switch">
                Already have an account? <a href="/login">Sign in</a>
            </p>
        </div>
    </div>
</body>
</html>

Overwriting templates/register.html


In [ ]:
%%writefile templates/home.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MediScan AI — Medical Report Simplifier</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="home-page">

    <!-- Animated Background Particles -->
    <div class="bg-particles">
        <div class="particle"></div><div class="particle"></div><div class="particle"></div>
        <div class="particle"></div><div class="particle"></div><div class="particle"></div>
    </div>

    <!-- Navbar -->
    <nav class="navbar">
        <div class="nav-brand">
            <span class="nav-icon">🏥</span>
            <span class="nav-title">MediScan <span class="highlight">AI</span></span>
        </div>
        <div class="nav-user">
            {% if user %}<span class="user-badge">👤 {{ user }}</span>{% endif %}
            <a href="/" class="nav-link active">Home</a>
            <a href="/analyze" class="nav-link">Analyze</a>
            <a href="/logout" class="logout-link">Logout</a>
        </div>
    </nav>

    <!-- Hero Section -->
    <section class="hero">
        <div class="hero-content">
            <div class="badge animate-fade-in">✨ Powered by GPT-4o Mini via OpenRouter</div>
            <h1 class="hero-title animate-slide-up">
                Understand Your<br>
                <span class="gradient-text">Medical Reports</span><br>
                Instantly
            </h1>
            <p class="hero-subtitle animate-slide-up delay-1">
                Upload your medical reports — lab results, prescriptions, scans — and get a
                <strong>clear, simple explanation</strong> in seconds. No medical degree required. 🧬
            </p>
            <div class="hero-actions animate-slide-up delay-2">
                <a href="/analyze" class="btn-primary">
                    <span>🔬 Analyze My Report</span>
                    <svg width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2"><path d="M5 12h14M12 5l7 7-7 7"/></svg>
                </a>
                <a href="#how-it-works" class="btn-secondary">How it Works</a>
            </div>

            <!-- Stats Bar -->
            <div class="stats-bar animate-fade-in delay-3">
                <div class="stat-item">
                    <span class="stat-num">📄</span>
                    <span class="stat-label">PDF / Image Support</span>
                </div>
                <div class="stat-divider"></div>
                <div class="stat-item">
                    <span class="stat-num">⚡</span>
                    <span class="stat-label">Instant Analysis</span>
                </div>
                <div class="stat-divider"></div>
                <div class="stat-item">
                    <span class="stat-num">🔒</span>
                    <span class="stat-label">Secure & Private</span>
                </div>
                <div class="stat-divider"></div>
                <div class="stat-item">
                    <span class="stat-num">🌐</span>
                    <span class="stat-label">Free to Use</span>
                </div>
            </div>
        </div>

        <!-- Hero Visual -->
        <div class="hero-visual animate-float">
            <div class="report-card">
                <div class="report-card-header">
                    <span>📋 Lab Report</span>
                    <span class="report-badge">Analyzing...</span>
                </div>
                <div class="report-lines">
                    <div class="report-line long pulse-line"></div>
                    <div class="report-line medium pulse-line delay-1"></div>
                    <div class="report-line short pulse-line delay-2"></div>
                    <div class="report-line long pulse-line delay-1"></div>
                    <div class="report-line medium pulse-line"></div>
                </div>
                <div class="report-card-footer">
                    <span class="ai-badge">🤖 AI Processing</span>
                </div>
            </div>
            <div class="arrow-down">↓</div>
            <div class="result-card">
                <div class="result-item">
                    <span class="result-icon">⚠️</span>
                    <div>
                        <div class="result-label">High Glucose</div>
                        <div class="result-val">Possible pre-diabetes</div>
                    </div>
                </div>
                <div class="result-item green">
                    <span class="result-icon">✅</span>
                    <div>
                        <div class="result-label">Hemoglobin</div>
                        <div class="result-val">Normal range</div>
                    </div>
                </div>
                <div class="result-item orange">
                    <span class="result-icon">🔺</span>
                    <div>
                        <div class="result-label">Cholesterol</div>
                        <div class="result-val">Slightly elevated</div>
                    </div>
                </div>
            </div>
        </div>
    </section>

    <!-- How It Works -->
    <section class="how-it-works" id="how-it-works">
        <div class="section-header">
            <h2>How It <span class="gradient-text">Works</span></h2>
            <p>Three simple steps to understand your health 🩺</p>
        </div>
        <div class="steps-grid">
            <div class="step-card">
                <div class="step-number">01</div>
                <div class="step-icon">📤</div>
                <h3>Upload Your Report</h3>
                <p>Upload any medical report — PDFs, images (JPG/PNG), or text files. We support scanned documents too via OCR.</p>
            </div>
            <div class="step-card">
                <div class="step-number">02</div>
                <div class="step-icon">🤖</div>
                <h3>AI Analyzes It</h3>
                <p>Our AI powered by GPT-4o Mini reads and understands your report, identifying abnormal findings and key data points.</p>
            </div>
            <div class="step-card">
                <div class="step-number">03</div>
                <div class="step-icon">💡</div>
                <h3>Get Simple Results</h3>
                <p>Receive a clear, jargon-free breakdown with findings, possible conditions, severity, and recommended next steps.</p>
            </div>
        </div>
    </section>

    <!-- Features Section -->
    <section class="features-section">
        <div class="section-header">
            <h2>Why Choose <span class="gradient-text">MediScan AI</span>?</h2>
            <p>Built with care for patients, not just professionals 💙</p>
        </div>
        <div class="features-grid">
            <div class="feature-card">
                <div class="feature-icon">🔬</div>
                <h4>Multi-Format Support</h4>
                <p>Works with PDFs, JPG, PNG, and TXT. Even scanned reports work via OCR technology.</p>
            </div>
            <div class="feature-card">
                <div class="feature-icon">🧠</div>
                <h4>AI-Powered Analysis</h4>
                <p>Uses state-of-the-art GPT-4o Mini for accurate interpretation of medical terminology.</p>
            </div>
            <div class="feature-card">
                <div class="feature-icon">📊</div>
                <h4>Structured Results</h4>
                <p>Get findings, explanations, severity ratings, and next steps — all neatly organized.</p>
            </div>
            <div class="feature-card">
                <div class="feature-icon">🔒</div>
                <h4>Privacy First</h4>
                <p>Your files are processed securely. We don't store or share your medical data.</p>
            </div>
            <div class="feature-card">
                <div class="feature-icon">⚡</div>
                <h4>Lightning Fast</h4>
                <p>Get your simplified report in seconds, not hours. No waiting, no appointments.</p>
            </div>
            <div class="feature-card">
                <div class="feature-icon">🌍</div>
                <h4>Accessible to All</h4>
                <p>No medical background needed. Written in plain language everyone can understand.</p>
            </div>
        </div>
    </section>

    <!-- Supported Reports -->
    <section class="reports-section">
        <div class="section-header">
            <h2>Supported Report <span class="gradient-text">Types</span></h2>
            <p>From routine checkups to specialized scans 🏥</p>
        </div>
        <div class="reports-grid">
            <div class="report-type-card">🧪 Blood Tests</div>
            <div class="report-type-card">🫀 ECG Reports</div>
            <div class="report-type-card">🦷 Dental Reports</div>
            <div class="report-type-card">🧠 MRI / CT Scans</div>
            <div class="report-type-card">💉 Pathology Reports</div>
            <div class="report-type-card">🩺 General Checkup</div>
            <div class="report-type-card">👁️ Eye Reports</div>
            <div class="report-type-card">💊 Prescriptions</div>
        </div>
    </section>

    <!-- CTA Section -->
    <section class="cta-section">
        <div class="cta-content">
            <h2>Ready to Understand Your Health? 🚀</h2>
            <p>Upload your medical report now and get a plain-language explanation in seconds.</p>
            <a href="/analyze" class="btn-primary large">
                <span>🔬 Start Analyzing Now</span>
                <svg width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2"><path d="M5 12h14M12 5l7 7-7 7"/></svg>
            </a>
            <p class="cta-disclaimer">⚕️ Not a substitute for professional medical advice. Always consult your doctor.</p>
        </div>
    </section>

    <!-- Footer -->
    <footer class="footer">
        <div class="footer-content">
            <div class="footer-brand">
                <span>🏥 MediScan AI</span>
                <p>Making healthcare understandable for everyone.</p>
            </div>
            <div class="footer-links">
                <a href="/">Home</a>
                <a href="/analyze">Analyze Report</a>
            </div>
        </div>
        <div class="footer-bottom">
            <p>⚠️ MediScan AI is for educational purposes only. It is not a diagnostic tool and should not replace professional medical advice.</p>
        </div>
    </footer>

    <script src="{{ url_for('static', filename='main.js') }}"></script>
</body>
</html>

Overwriting templates/home.html


In [ ]:
%%writefile templates/analyze.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Analyze Report — MediScan AI</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="analyze-page">

    <!-- Animated Background -->
    <div class="bg-particles">
        <div class="particle"></div><div class="particle"></div><div class="particle"></div>
        <div class="particle"></div><div class="particle"></div>
    </div>

    <!-- Navbar -->
    <nav class="navbar">
        <div class="nav-brand">
            <a href="/" style="text-decoration:none; display:flex; align-items:center; gap:8px;">
                <span class="nav-icon">🏥</span>
                <span class="nav-title">MediScan <span class="highlight">AI</span></span>
            </a>
        </div>
        <div class="nav-links">
            <a href="/" class="nav-link">Home</a>
            <a href="/analyze" class="nav-link active">Analyze</a>
        </div>
    </nav>

    <!-- Page Header -->
    <div class="page-header">
        <div class="page-header-content">
            <div class="badge">🔬 AI-Powered Analysis Engine</div>
            <h1>Upload Your <span class="gradient-text">Medical Report</span></h1>
            <p>Securely upload your file and our AI will decode it for you in seconds</p>
        </div>
    </div>

    <!-- Main Upload Section -->
    <div class="analyze-container">

        <!-- Left: Upload Form -->
        <div class="upload-section">
            <form method="post" enctype="multipart/form-data" id="uploadForm" onsubmit="handleSubmit(event)">

                <!-- Drop Zone -->
                <div class="drop-zone" id="dropZone" onclick="document.getElementById('fileInput').click()">
                    <div class="drop-zone-content">
                        <div class="upload-icon animate-bounce">📂</div>
                        <h3>Drop your file here</h3>
                        <p>or <span class="click-text">click to browse</span></p>
                        <div class="file-formats">
                            <span class="format-tag">📄 PDF</span>
                            <span class="format-tag">🖼️ JPG</span>
                            <span class="format-tag">🖼️ PNG</span>
                            <span class="format-tag">📝 TXT</span>
                        </div>
                    </div>
                    <input type="file" name="report" id="fileInput" accept=".pdf,.jpg,.jpeg,.png,.txt" required style="display:none" onchange="handleFileSelect(this)">
                </div>

                <!-- File Preview -->
                <div class="file-preview" id="filePreview" style="display:none">
                    <div class="file-preview-content">
                        <span class="file-icon" id="fileIcon">📄</span>
                        <div class="file-info">
                            <div class="file-name" id="fileName">report.pdf</div>
                            <div class="file-size" id="fileSize">2.4 MB</div>
                        </div>
                        <button type="button" class="remove-file" onclick="removeFile()">✕</button>
                    </div>
                    <div class="file-progress">
                        <div class="file-progress-bar"></div>
                    </div>
                </div>

                {% if error %}
                <div class="error-alert">
                    <span>⚠️</span>
                    <span>{{ error }}</span>
                </div>
                {% endif %}

                <!-- Submit Button -->
                <button type="submit" class="btn-analyze" id="analyzeBtn">
                    <span id="btnText">🧠 Analyze My Report</span>
                    <div class="btn-loader" id="btnLoader" style="display:none">
                        <div class="loader-dots">
                            <div></div><div></div><div></div>
                        </div>
                        <span>Analyzing...</span>
                    </div>
                </button>

            </form>

            <!-- Privacy Note -->
            <div class="privacy-note">
                <span>🔒</span>
                <span>Your file is processed securely. We don't store your medical data after analysis.</span>
            </div>
        </div>

        <!-- Right: Info Panel -->
        <div class="info-panel">
            <div class="info-card">
                <h3>🤖 What our AI Analyzes</h3>
                <ul class="info-list">
                    <li>⚠️ Abnormal values & findings</li>
                    <li>💬 Plain-language explanations</li>
                    <li>🏥 Possible medical conditions</li>
                    <li>📊 Severity assessment</li>
                    <li>🩺 Recommended next steps</li>
                </ul>
            </div>

            <div class="info-card">
                <h3>📋 Supported File Types</h3>
                <div class="file-type-grid">
                    <div class="file-type-item">
                        <span>📄</span>
                        <div>
                            <strong>PDF</strong>
                            <small>Lab reports, prescriptions</small>
                        </div>
                    </div>
                    <div class="file-type-item">
                        <span>🖼️</span>
                        <div>
                            <strong>JPG / PNG</strong>
                            <small>Scanned or photo reports</small>
                        </div>
                    </div>
                    <div class="file-type-item">
                        <span>📝</span>
                        <div>
                            <strong>TXT</strong>
                            <small>Plain text reports</small>
                        </div>
                    </div>
                </div>
            </div>

            <div class="info-card tips-card">
                <h3>💡 Tips for Best Results</h3>
                <p>✅ Use clear, high-resolution scans</p>
                <p>✅ Ensure text is readable in images</p>
                <p>✅ Upload complete reports (all pages)</p>
                <p>✅ Include reference ranges if available</p>
            </div>

            <div class="disclaimer-card">
                <span>⚕️</span>
                <p>This tool is <strong>not a diagnostic tool</strong>. Results are for educational purposes only. Always consult a qualified medical professional for diagnosis and treatment.</p>
            </div>
        </div>

    </div>

    <!-- Loading Overlay -->
    <div class="loading-overlay" id="loadingOverlay" style="display:none">
        <div class="loading-content">
            <div class="loading-animation">
                <div class="pulse-ring"></div>
                <div class="pulse-ring delay-1"></div>
                <div class="pulse-ring delay-2"></div>
                <span class="loading-icon">🧬</span>
            </div>
            <h3>Analyzing Your Report...</h3>
            <p id="loadingMsg">Extracting text from your file</p>
            <div class="loading-steps">
                <div class="loading-step active" id="step1">📤 File Uploaded</div>
                <div class="loading-step" id="step2">📖 Reading Content</div>
                <div class="loading-step" id="step3">🤖 AI Processing</div>
                <div class="loading-step" id="step4">✨ Generating Report</div>
            </div>
        </div>
    </div>

    <script src="{{ url_for('static', filename='main.js') }}"></script>
</body>
</html>

Overwriting templates/analyze.html


In [ ]:
%%writefile templates/result.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Analysis Results — MediScan AI</title>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700;800&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body class="result-page">

    <!-- Animated Background -->
    <div class="bg-particles">
        <div class="particle"></div><div class="particle"></div><div class="particle"></div>
    </div>

    <!-- Navbar -->
    <nav class="navbar">
        <div class="nav-brand">
            <a href="/" style="text-decoration:none; display:flex; align-items:center; gap:8px;">
                <span class="nav-icon">🏥</span>
                <span class="nav-title">MediScan <span class="highlight">AI</span></span>
            </a>
        </div>
        <div class="nav-links">
            <a href="/" class="nav-link">Home</a>
            <a href="/analyze" class="nav-link">Analyze</a>
        </div>
    </nav>

    <!-- Result Header -->
    <div class="result-header">
        <div class="result-header-content">
            <div class="success-badge animate-pop">✅ Analysis Complete</div>
            <h1>Your Report <span class="gradient-text">Results</span></h1>
            <p>AI has analyzed <strong>{{ data.filename }}</strong> — here's what it found 🔍</p>
        </div>
    </div>

    <!-- Results Container -->
    <div class="results-container">

        <!-- Summary Cards Row -->
        <div class="summary-row animate-slide-up">
            <div class="summary-card red">
                <div class="summary-card-icon">⚠️</div>
                <div class="summary-card-label">Abnormal Findings</div>
            </div>
            <div class="summary-card blue">
                <div class="summary-card-icon">💬</div>
                <div class="summary-card-label">Simplified</div>
            </div>
            <div class="summary-card purple">
                <div class="summary-card-icon">🏥</div>
                <div class="summary-card-label">Conditions Found</div>
            </div>
            <div class="summary-card orange">
                <div class="summary-card-icon">📊</div>
                <div class="summary-card-label">Severity Rated</div>
            </div>
        </div>

        <!-- Main Results Grid -->
        <div class="results-grid">

            <!-- Abnormal Findings -->
            {% if data.abnormal_findings %}
            <div class="result-section-card abnormal animate-card" style="animation-delay: 0.1s">
                <div class="section-card-header">
                    <div class="section-icon-wrap red-bg">⚠️</div>
                    <div>
                        <h3>Abnormal Findings</h3>
                        <p>Values or results outside normal range</p>
                    </div>
                </div>
                <div class="section-card-body">
                    {{ data.abnormal_findings | replace('\n', '<br>') | safe }}
                </div>
            </div>
            {% endif %}

            <!-- Simple Explanation -->
            {% if data.simple_explanation %}
            <div class="result-section-card explanation animate-card" style="animation-delay: 0.2s">
                <div class="section-card-header">
                    <div class="section-icon-wrap blue-bg">💬</div>
                    <div>
                        <h3>Simple Explanation</h3>
                        <p>What these findings mean in plain English</p>
                    </div>
                </div>
                <div class="section-card-body">
                    {{ data.simple_explanation | replace('\n', '<br>') | safe }}
                </div>
            </div>
            {% endif %}

            <!-- Possible Conditions -->
            {% if data.possible_conditions %}
            <div class="result-section-card conditions animate-card" style="animation-delay: 0.3s">
                <div class="section-card-header">
                    <div class="section-icon-wrap purple-bg">🏥</div>
                    <div>
                        <h3>Possible Conditions</h3>
                        <p>Not a diagnosis — consult your doctor</p>
                    </div>
                </div>
                <div class="section-card-body">
                    {{ data.possible_conditions | replace('\n', '<br>') | safe }}
                </div>
            </div>
            {% endif %}

            <!-- Severity -->
            {% if data.severity %}
            <div class="result-section-card severity animate-card" style="animation-delay: 0.4s">
                <div class="section-card-header">
                    <div class="section-icon-wrap orange-bg">📊</div>
                    <div>
                        <h3>Severity Assessment</h3>
                        <p>How urgent is attention needed</p>
                    </div>
                </div>
                <div class="section-card-body">
                    {{ data.severity | replace('\n', '<br>') | safe }}
                </div>
            </div>
            {% endif %}

            <!-- Next Steps -->
            {% if data.next_steps %}
            <div class="result-section-card next-steps animate-card" style="animation-delay: 0.5s">
                <div class="section-card-header">
                    <div class="section-icon-wrap green-bg">🩺</div>
                    <div>
                        <h3>Recommended Next Steps</h3>
                        <p>What you should do now</p>
                    </div>
                </div>
                <div class="section-card-body">
                    {{ data.next_steps | replace('\n', '<br>') | safe }}
                </div>
            </div>
            {% endif %}

            <!-- Full Raw Response (if sections not parsed) -->
            {% if not data.abnormal_findings and not data.simple_explanation %}
            <div class="result-section-card full-report animate-card" style="animation-delay: 0.1s; grid-column: 1/-1">
                <div class="section-card-header">
                    <div class="section-icon-wrap blue-bg">📋</div>
                    <div>
                        <h3>Full AI Analysis</h3>
                        <p>Complete breakdown of your medical report</p>
                    </div>
                </div>
                <div class="section-card-body">
                    {{ data.raw | replace('\n', '<br>') | safe }}
                </div>
            </div>
            {% endif %}

        </div>

        <!-- Disclaimer Card -->
        {% if data.disclaimer %}
        <div class="result-disclaimer animate-card" style="animation-delay: 0.6s">
            <div class="disclaimer-icon">⚕️</div>
            <div class="disclaimer-text">
                <strong>Medical Disclaimer</strong>
                <p>{{ data.disclaimer }}</p>
            </div>
        </div>
        {% else %}
        <div class="result-disclaimer animate-card" style="animation-delay: 0.6s">
            <div class="disclaimer-icon">⚕️</div>
            <div class="disclaimer-text">
                <strong>Medical Disclaimer</strong>
                <p>This analysis is generated by AI and is for <strong>informational purposes only</strong>. It is NOT a medical diagnosis. Always consult a qualified healthcare professional before making any medical decisions.</p>
            </div>
        </div>
        {% endif %}

        <!-- Action Buttons -->
        <div class="result-actions animate-card" style="animation-delay: 0.7s">
            <a href="/analyze" class="btn-primary">
                <span>🔬 Analyze Another Report</span>
            </a>
            <button onclick="window.print()" class="btn-secondary">
                🖨️ Print Results
            </button>
            <a href="/" class="btn-ghost">
                🏠 Back to Home
            </a>
        </div>

        <!-- Share Note -->
        <p class="share-note">💡 Pro tip: Take a screenshot or print these results to share with your doctor for a more informed consultation.</p>

    </div>

    <script src="{{ url_for('static', filename='main.js') }}"></script>
</body>
</html>

Overwriting templates/result.html


In [ ]:
%%writefile static/style.css
/* ============================================
   MEDISCAN AI — Global Styles
   ============================================ */

*, *::before, *::after { box-sizing: border-box; margin: 0; padding: 0; }

:root {
    --primary: #6C63FF;
    --primary-dark: #5a52d5;
    --secondary: #00D4FF;
    --accent: #FF6B9D;
    --success: #00E676;
    --warning: #FFB300;
    --danger: #FF5252;
    --bg-dark: #0A0E1A;
    --bg-card: rgba(255,255,255,0.05);
    --bg-card-hover: rgba(255,255,255,0.08);
    --border: rgba(255,255,255,0.1);
    --text-primary: #FFFFFF;
    --text-secondary: rgba(255,255,255,0.65);
    --text-muted: rgba(255,255,255,0.4);
    --gradient: linear-gradient(135deg, #6C63FF, #00D4FF);
    --gradient-warm: linear-gradient(135deg, #FF6B9D, #FFB300);
    --shadow-lg: 0 20px 60px rgba(0,0,0,0.5);
    --shadow-glow: 0 0 40px rgba(108,99,255,0.3);
    --radius: 16px;
    --radius-sm: 10px;
}

html { scroll-behavior: smooth; }

body {
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, sans-serif;
    background: var(--bg-dark);
    color: var(--text-primary);
    min-height: 100vh;
    overflow-x: hidden;
}

/* ============================================
   BACKGROUND PARTICLES
   ============================================ */

.bg-particles {
    position: fixed;
    inset: 0;
    pointer-events: none;
    z-index: 0;
    overflow: hidden;
}

.particle {
    position: absolute;
    border-radius: 50%;
    animation: float-particle 15s infinite ease-in-out;
    opacity: 0.08;
}

.particle:nth-child(1) { width:300px; height:300px; background:var(--primary); top:-50px; left:-100px; animation-duration:18s; }
.particle:nth-child(2) { width:200px; height:200px; background:var(--secondary); top:40%; right:-50px; animation-duration:22s; animation-delay:3s; }
.particle:nth-child(3) { width:150px; height:150px; background:var(--accent); bottom:10%; left:20%; animation-duration:16s; animation-delay:6s; }
.particle:nth-child(4) { width:250px; height:250px; background:var(--primary); bottom:-80px; right:30%; animation-duration:20s; animation-delay:2s; }
.particle:nth-child(5) { width:100px; height:100px; background:var(--success); top:30%; left:40%; animation-duration:14s; animation-delay:8s; }
.particle:nth-child(6) { width:180px; height:180px; background:var(--secondary); top:70%; left:60%; animation-duration:19s; animation-delay:4s; }

@keyframes float-particle {
    0%,100% { transform: translate(0,0) scale(1); }
    33% { transform: translate(30px,-40px) scale(1.1); }
    66% { transform: translate(-20px,20px) scale(0.9); }
}

/* ============================================
   NAVBAR
   ============================================ */

.navbar {
    position: fixed;
    top: 0; left: 0; right: 0;
    z-index: 100;
    display: flex;
    align-items: center;
    justify-content: space-between;
    padding: 16px 40px;
    background: rgba(10,14,26,0.8);
    backdrop-filter: blur(20px);
    border-bottom: 1px solid var(--border);
}

.nav-brand {
    display: flex;
    align-items: center;
    gap: 10px;
}

.nav-icon { font-size: 1.5rem; }

.nav-title {
    font-size: 1.25rem;
    font-weight: 700;
    color: var(--text-primary);
    text-decoration: none;
}

.nav-title .highlight { color: var(--primary); }

.nav-links { display: flex; gap: 8px; }

.nav-link {
    padding: 8px 20px;
    border-radius: 50px;
    color: var(--text-secondary);
    text-decoration: none;
    font-size: 0.9rem;
    font-weight: 500;
    transition: all 0.3s;
}

.nav-link:hover { color: var(--text-primary); background: var(--bg-card); }
.nav-link.active { color: var(--text-primary); background: var(--primary); }

/* ============================================
   UTILITY CLASSES
   ============================================ */

.gradient-text {
    background: var(--gradient);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;
}

.badge {
    display: inline-flex;
    align-items: center;
    gap: 6px;
    padding: 6px 16px;
    background: rgba(108,99,255,0.15);
    border: 1px solid rgba(108,99,255,0.3);
    border-radius: 50px;
    font-size: 0.8rem;
    font-weight: 500;
    color: var(--primary);
    margin-bottom: 20px;
}

/* ============================================
   BUTTONS
   ============================================ */

.btn-primary {
    display: inline-flex;
    align-items: center;
    gap: 10px;
    padding: 14px 28px;
    background: var(--gradient);
    border: none;
    border-radius: 50px;
    color: white;
    font-size: 1rem;
    font-weight: 600;
    cursor: pointer;
    text-decoration: none;
    transition: all 0.3s;
    box-shadow: 0 8px 30px rgba(108,99,255,0.4);
}

.btn-primary:hover {
    transform: translateY(-2px);
    box-shadow: 0 12px 40px rgba(108,99,255,0.6);
}

.btn-primary.large { padding: 18px 36px; font-size: 1.1rem; }

.btn-secondary {
    display: inline-flex;
    align-items: center;
    gap: 8px;
    padding: 14px 28px;
    background: transparent;
    border: 1px solid var(--border);
    border-radius: 50px;
    color: var(--text-secondary);
    font-size: 0.95rem;
    font-weight: 500;
    cursor: pointer;
    text-decoration: none;
    transition: all 0.3s;
}

.btn-secondary:hover { border-color: var(--primary); color: var(--primary); background: rgba(108,99,255,0.1); }

.btn-ghost {
    display: inline-flex;
    align-items: center;
    gap: 8px;
    padding: 14px 28px;
    background: transparent;
    border: none;
    color: var(--text-muted);
    font-size: 0.9rem;
    cursor: pointer;
    text-decoration: none;
    transition: all 0.3s;
    border-radius: 50px;
}

.btn-ghost:hover { color: var(--text-primary); background: var(--bg-card); }

/* ============================================
   ANIMATIONS
   ============================================ */

@keyframes fade-in { from { opacity:0; } to { opacity:1; } }
@keyframes slide-up { from { opacity:0; transform:translateY(40px); } to { opacity:1; transform:translateY(0); } }
@keyframes pop { from { opacity:0; transform:scale(0.8); } to { opacity:1; transform:scale(1); } }
@keyframes float { 0%,100% { transform:translateY(0); } 50% { transform:translateY(-20px); } }
@keyframes bounce-icon { 0%,100% { transform:translateY(0); } 50% { transform:translateY(-8px); } }
@keyframes pulse-glow { 0%,100% { opacity:0.3; transform:scale(1); } 50% { opacity:0.6; transform:scale(1.1); } }
@keyframes shimmer { 0% { background-position:-200% 0; } 100% { background-position:200% 0; } }
@keyframes card-in { from { opacity:0; transform:translateY(30px) scale(0.97); } to { opacity:1; transform:translateY(0) scale(1); } }

.animate-fade-in { animation: fade-in 0.8s ease both; }
.animate-slide-up { animation: slide-up 0.8s ease both; }
.animate-pop { animation: pop 0.5s cubic-bezier(0.34,1.56,0.64,1) both; }
.animate-float { animation: float 4s ease-in-out infinite; }
.animate-bounce { animation: bounce-icon 2s ease-in-out infinite; }
.animate-card { animation: card-in 0.6s ease both; }

.delay-1 { animation-delay: 0.15s; }
.delay-2 { animation-delay: 0.3s; }
.delay-3 { animation-delay: 0.45s; }
.delay-4 { animation-delay: 0.6s; }

/* ============================================
   SECTION HEADERS
   ============================================ */

.section-header {
    text-align: center;
    margin-bottom: 50px;
}

.section-header h2 {
    font-size: 2.2rem;
    font-weight: 800;
    margin-bottom: 12px;
}

.section-header p {
    color: var(--text-secondary);
    font-size: 1.05rem;
}

/* ============================================
   HOME PAGE — HERO
   ============================================ */

.home-page { padding-top: 80px; }

.hero {
    display: flex;
    align-items: center;
    gap: 60px;
    max-width: 1200px;
    margin: 0 auto;
    padding: 80px 40px;
    position: relative;
    z-index: 1;
}

.hero-content { flex: 1; }

.hero-title {
    font-size: clamp(2.5rem, 5vw, 3.8rem);
    font-weight: 800;
    line-height: 1.15;
    margin-bottom: 20px;
}

.hero-subtitle {
    font-size: 1.1rem;
    color: var(--text-secondary);
    line-height: 1.7;
    margin-bottom: 35px;
    max-width: 480px;
}

.hero-actions {
    display: flex;
    gap: 16px;
    align-items: center;
    flex-wrap: wrap;
    margin-bottom: 40px;
}

/* Stats Bar */
.stats-bar {
    display: flex;
    gap: 0;
    background: var(--bg-card);
    border: 1px solid var(--border);
    border-radius: var(--radius);
    padding: 16px 24px;
    max-width: 520px;
}

.stat-item {
    display: flex;
    flex-direction: column;
    align-items: center;
    gap: 4px;
    flex: 1;
    text-align: center;
}

.stat-num { font-size: 1.3rem; }
.stat-label { font-size: 0.7rem; color: var(--text-muted); font-weight: 500; }
.stat-divider { width: 1px; background: var(--border); margin: 0 8px; }

/* Hero Visual */
.hero-visual {
    flex: 0 0 360px;
    display: flex;
    flex-direction: column;
    align-items: center;
    gap: 16px;
    position: relative;
    z-index: 1;
}

.report-card, .result-card {
    background: var(--bg-card);
    border: 1px solid var(--border);
    border-radius: var(--radius);
    padding: 20px;
    width: 300px;
    backdrop-filter: blur(10px);
}

.report-card-header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-bottom: 16px;
    font-size: 0.9rem;
    font-weight: 600;
}

.report-badge {
    font-size: 0.7rem;
    background: rgba(0,212,255,0.15);
    color: var(--secondary);
    padding: 4px 10px;
    border-radius: 50px;
    border: 1px solid rgba(0,212,255,0.3);
}

.report-lines { display: flex; flex-direction: column; gap: 10px; }
.report-line { height: 10px; border-radius: 5px; background: rgba(255,255,255,0.1); }
.report-line.long { width: 100%; }
.report-line.medium { width: 70%; }
.report-line.short { width: 45%; }

.pulse-line {
    background: linear-gradient(90deg, rgba(255,255,255,0.05), rgba(255,255,255,0.2), rgba(255,255,255,0.05));
    background-size: 200% 100%;
    animation: shimmer 2s infinite;
}

.report-card-footer { margin-top: 16px; text-align: center; }
.ai-badge {
    font-size: 0.75rem;
    background: rgba(108,99,255,0.2);
    color: var(--primary);
    padding: 5px 12px;
    border-radius: 50px;
    border: 1px solid rgba(108,99,255,0.3);
}

.arrow-down {
    font-size: 1.5rem;
    color: var(--primary);
    animation: bounce-icon 2s ease-in-out infinite;
}

.result-item {
    display: flex;
    align-items: center;
    gap: 12px;
    padding: 10px;
    border-radius: var(--radius-sm);
    background: rgba(255,82,82,0.1);
    border: 1px solid rgba(255,82,82,0.2);
    margin-bottom: 8px;
}

.result-item.green { background: rgba(0,230,118,0.1); border-color: rgba(0,230,118,0.2); }
.result-item.orange { background: rgba(255,179,0,0.1); border-color: rgba(255,179,0,0.2); }

.result-icon { font-size: 1.1rem; }
.result-label { font-size: 0.85rem; font-weight: 600; }
.result-val { font-size: 0.75rem; color: var(--text-secondary); }

/* ============================================
   HOME PAGE — HOW IT WORKS
   ============================================ */

.how-it-works {
    max-width: 1100px;
    margin: 0 auto;
    padding: 80px 40px;
    position: relative;
    z-index: 1;
}

.steps-grid {
    display: grid;
    grid-template-columns: repeat(3, 1fr);
    gap: 24px;
}

.step-card {
    background: var(--bg-card);
    border: 1px solid var(--border);
    border-radius: var(--radius);
    padding: 32px;
    position: relative;
    transition: all 0.3s;
}

.step-card:hover {
    border-color: var(--primary);
    background: var(--bg-card-hover);
    transform: translateY(-4px);
    box-shadow: var(--shadow-glow);
}

.step-number {
    position: absolute;
    top: 20px;
    right: 20px;
    font-size: 2.5rem;
    font-weight: 900;
    background: var(--gradient);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    background-clip: text;
    opacity: 0.3;
}

.step-icon { font-size: 2.5rem; margin-bottom: 16px; display: block; }
.step-card h3 { font-size: 1.15rem; font-weight: 700; margin-bottom: 10px; }
.step-card p { color: var(--text-secondary); font-size: 0.9rem; line-height: 1.6; }

/* ============================================
   HOME PAGE — FEATURES
   ============================================ */

.features-section {
    max-width: 1100px;
    margin: 0 auto;
    padding: 60px 40px;
    position: relative;
    z-index: 1;
}

.features-grid {
    display: grid;
    grid-template-columns: repeat(3, 1fr);
    gap: 20px;
}

.feature-card {
    background: var(--bg-card);
    border: 1px solid var(--border);
    border-radius: var(--radius);
    padding: 28px;
    transition: all 0.3s;
}

.feature-card:hover { border-color: rgba(108,99,255,0.4); transform: translateY(-3px); }

.feature-icon { font-size: 2rem; margin-bottom: 14px; display: block; }
.feature-card h4 { font-size: 1rem; font-weight: 600; margin-bottom: 8px; }
.feature-card p { color: var(--text-secondary); font-size: 0.85rem; line-height: 1.6; }

/* ============================================
   HOME PAGE — SUPPORTED REPORTS
   ============================================ */

.reports-section {
    max-width: 1100px;
    margin: 0 auto;
    padding: 60px 40px;
    position: relative;
    z-index: 1;
}

.reports-grid {
    display: flex;
    flex-wrap: wrap;
    gap: 12px;
    justify-content: center;
}

.report-type-card {
    background: var(--bg-card);
    border: 1px solid var(--border);
    border-radius: 50px;
    padding: 12px 24px;
    font-size: 0.9rem;
    font-weight: 500;
    transition: all 0.3s;
    cursor: default;
}

.report-type-card:hover { border-color: var(--primary); color: var(--primary); background: rgba(108,99,255,0.1); }

/* ============================================
   HOME PAGE — CTA
   ============================================ */

.cta-section {
    max-width: 700px;
    margin: 40px auto;
    padding: 80px 40px;
    text-align: center;
    position: relative;
    z-index: 1;
}

.cta-content h2 { font-size: 2.2rem; font-weight: 800; margin-bottom: 16px; }
.cta-content p { color: var(--text-secondary); font-size: 1.05rem; margin-bottom: 35px; }
.cta-disclaimer { margin-top: 20px; font-size: 0.8rem; color: var(--text-muted); }

/* ============================================
   FOOTER
   ============================================ */

.footer {
    border-top: 1px solid var(--border);
    position: relative;
    z-index: 1;
    margin-top: 40px;
}

.footer-content {
    max-width: 1100px;
    margin: 0 auto;
    padding: 40px;
    display: flex;
    justify-content: space-between;
    align-items: center;
}

.footer-brand span { font-size: 1.1rem; font-weight: 700; }
.footer-brand p { color: var(--text-muted); font-size: 0.85rem; margin-top: 6px; }

.footer-links { display: flex; gap: 24px; }
.footer-links a { color: var(--text-secondary); text-decoration: none; font-size: 0.9rem; transition: color 0.3s; }
.footer-links a:hover { color: var(--primary); }

.footer-bottom {
    border-top: 1px solid var(--border);
    padding: 20px 40px;
    text-align: center;
}

.footer-bottom p { color: var(--text-muted); font-size: 0.8rem; }

/* ============================================
   ANALYZE PAGE
   ============================================ */

.analyze-page { padding-top: 80px; }

.page-header {
    position: relative;
    z-index: 1;
    padding: 60px 40px 40px;
    text-align: center;
    border-bottom: 1px solid var(--border);
    background: linear-gradient(to bottom, rgba(108,99,255,0.05), transparent);
}

.page-header-content h1 { font-size: 2.5rem; font-weight: 800; margin-bottom: 12px; }
.page-header-content p { color: var(--text-secondary); font-size: 1rem; }

.analyze-container {
    max-width: 1100px;
    margin: 0 auto;
    padding: 50px 40px;
    display: grid;
    grid-template-columns: 1.2fr 0.8fr;
    gap: 40px;
    position: relative;
    z-index: 1;
}

/* Drop Zone */
.upload-section { display: flex; flex-direction: column; gap: 20px; }

.drop-zone {
    border: 2px dashed var(--border);
    border-radius: var(--radius);
    padding: 60px 40px;
    text-align: center;
    cursor: pointer;
    transition: all 0.3s;
    background: var(--bg-card);
    position: relative;
}

.drop-zone:hover, .drop-zone.drag-over {
    border-color: var(--primary);
    background: rgba(108,99,255,0.08);
    transform: scale(1.01);
}

.drop-zone-content { pointer-events: none; }
.upload-icon { font-size: 3.5rem; display: block; margin-bottom: 16px; }
.drop-zone h3 { font-size: 1.2rem; font-weight: 600; margin-bottom: 8px; }
.drop-zone p { color: var(--text-secondary); margin-bottom: 20px; }
.click-text { color: var(--primary); font-weight: 600; text-decoration: underline; }

.file-formats { display: flex; gap: 8px; justify-content: center; flex-wrap: wrap; }
.format-tag {
    background: rgba(108,99,255,0.1);
    border: 1px solid rgba(108,99,255,0.3);
    color: var(--primary);
    padding: 5px 12px;
    border-radius: 50px;
    font-size: 0.8rem;
    font-weight: 500;
}

/* File Preview */
.file-preview {
    background: rgba(0,230,118,0.05);
    border: 1px solid rgba(0,230,118,0.2);
    border-radius: var(--radius);
    padding: 16px 20px;
    animation: slide-up 0.3s ease;
}

.file-preview-content { display: flex; align-items: center; gap: 14px; }
.file-icon { font-size: 2rem; }
.file-info { flex: 1; }
.file-name { font-weight: 600; font-size: 0.95rem; }
.file-size { color: var(--text-muted); font-size: 0.8rem; }

.remove-file {
    background: rgba(255,82,82,0.1);
    border: none;
    color: var(--danger);
    width: 30px;
    height: 30px;
    border-radius: 50%;
    cursor: pointer;
    font-size: 0.85rem;
    transition: all 0.3s;
}

.remove-file:hover { background: var(--danger); color: white; }

.file-progress {
    margin-top: 12px;
    height: 4px;
    background: rgba(255,255,255,0.1);
    border-radius: 2px;
    overflow: hidden;
}

.file-progress-bar {
    height: 100%;
    width: 100%;
    background: var(--gradient);
    border-radius: 2px;
    animation: shimmer 1.5s infinite;
    background-size: 200% 100%;
}

/* Error Alert */
.error-alert {
    display: flex;
    align-items: center;
    gap: 10px;
    padding: 14px 18px;
    background: rgba(255,82,82,0.1);
    border: 1px solid rgba(255,82,82,0.3);
    border-radius: var(--radius-sm);
    color: #ff8080;
    font-size: 0.9rem;
    animation: slide-up 0.3s ease;
}

/* Analyze Button */
.btn-analyze {
    width: 100%;
    padding: 18px;
    background: var(--gradient);
    border: none;
    border-radius: var(--radius);
    color: white;
    font-size: 1.1rem;
    font-weight: 700;
    cursor: pointer;
    transition: all 0.3s;
    box-shadow: 0 8px 30px rgba(108,99,255,0.4);
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 10px;
}

.btn-analyze:hover { transform: translateY(-2px); box-shadow: 0 12px 40px rgba(108,99,255,0.6); }
.btn-analyze:disabled { opacity: 0.6; cursor: not-allowed; transform: none; }

.loader-dots { display: flex; gap: 5px; align-items: center; }
.loader-dots div {
    width: 8px; height: 8px;
    background: white;
    border-radius: 50%;
    animation: bounce-icon 0.8s ease-in-out infinite;
}
.loader-dots div:nth-child(2) { animation-delay: 0.1s; }
.loader-dots div:nth-child(3) { animation-delay: 0.2s; }

/* Privacy Note */
.privacy-note {
    display: flex;
    align-items: center;
    gap: 8px;
    padding: 12px 16px;
    background: rgba(0,212,255,0.05);
    border: 1px solid rgba(0,212,255,0.15);
    border-radius: var(--radius-sm);
    font-size: 0.8rem;
    color: var(--text-muted);
}

/* Info Panel */
.info-panel { display: flex; flex-direction: column; gap: 20px; }

.info-card {
    background: var(--bg-card);
    border: 1px solid var(--border);
    border-radius: var(--radius);
    padding: 22px;
}

.info-card h3 { font-size: 0.95rem; font-weight: 700; margin-bottom: 14px; }

.info-list { list-style: none; display: flex; flex-direction: column; gap: 8px; }
.info-list li { color: var(--text-secondary); font-size: 0.85rem; padding: 6px 0; border-bottom: 1px solid var(--border); }
.info-list li:last-child { border-bottom: none; }

.file-type-grid { display: flex; flex-direction: column; gap: 12px; }
.file-type-item { display: flex; align-items: center; gap: 12px; }
.file-type-item span { font-size: 1.5rem; }
.file-type-item strong { display: block; font-size: 0.85rem; }
.file-type-item small { color: var(--text-muted); font-size: 0.75rem; }

.tips-card p { font-size: 0.85rem; color: var(--text-secondary); margin-bottom: 8px; }
.tips-card p:last-child { margin-bottom: 0; }

.disclaimer-card {
    display: flex;
    gap: 12px;
    padding: 16px;
    background: rgba(255,179,0,0.05);
    border: 1px solid rgba(255,179,0,0.2);
    border-radius: var(--radius-sm);
    font-size: 0.8rem;
    color: var(--text-muted);
}

.disclaimer-card span { font-size: 1.1rem; flex-shrink: 0; }

/* ============================================
   LOADING OVERLAY
   ============================================ */

.loading-overlay {
    position: fixed;
    inset: 0;
    z-index: 1000;
    background: rgba(10,14,26,0.95);
    backdrop-filter: blur(10px);
    display: flex;
    align-items: center;
    justify-content: center;
}

.loading-content { text-align: center; }

.loading-animation {
    position: relative;
    width: 100px;
    height: 100px;
    margin: 0 auto 30px;
    display: flex;
    align-items: center;
    justify-content: center;
}

.pulse-ring {
    position: absolute;
    inset: 0;
    border-radius: 50%;
    border: 2px solid var(--primary);
    animation: pulse-glow 2s ease-in-out infinite;
}

.pulse-ring.delay-1 { animation-delay: 0.5s; transform: scale(1.2); opacity: 0.5; }
.pulse-ring.delay-2 { animation-delay: 1s; transform: scale(1.4); opacity: 0.3; }

.loading-icon { font-size: 2.5rem; position: relative; z-index: 1; }

.loading-content h3 { font-size: 1.4rem; font-weight: 700; margin-bottom: 10px; }
.loading-content p { color: var(--text-secondary); font-size: 0.95rem; margin-bottom: 30px; }

.loading-steps {
    display: flex;
    gap: 8px;
    justify-content: center;
    flex-wrap: wrap;
}

.loading-step {
    padding: 8px 16px;
    border-radius: 50px;
    font-size: 0.8rem;
    background: var(--bg-card);
    border: 1px solid var(--border);
    color: var(--text-muted);
    transition: all 0.4s;
}

.loading-step.active {
    background: rgba(108,99,255,0.15);
    border-color: var(--primary);
    color: var(--primary);
}

/* ============================================
   RESULT PAGE
   ============================================ */

.result-page { padding-top: 80px; }

.result-header {
    position: relative;
    z-index: 1;
    padding: 60px 40px 40px;
    text-align: center;
    background: linear-gradient(to bottom, rgba(0,230,118,0.04), transparent);
    border-bottom: 1px solid var(--border);
}

.result-header-content h1 { font-size: 2.5rem; font-weight: 800; margin-bottom: 12px; }
.result-header-content p { color: var(--text-secondary); font-size: 1rem; }

.success-badge {
    display: inline-flex;
    align-items: center;
    gap: 6px;
    padding: 6px 18px;
    background: rgba(0,230,118,0.1);
    border: 1px solid rgba(0,230,118,0.3);
    border-radius: 50px;
    font-size: 0.85rem;
    font-weight: 600;
    color: var(--success);
    margin-bottom: 20px;
}

.results-container {
    max-width: 1100px;
    margin: 0 auto;
    padding: 50px 40px;
    position: relative;
    z-index: 1;
}

/* Summary Row */
.summary-row {
    display: grid;
    grid-template-columns: repeat(4, 1fr);
    gap: 16px;
    margin-bottom: 40px;
}

.summary-card {
    background: var(--bg-card);
    border: 1px solid var(--border);
    border-radius: var(--radius);
    padding: 20px;
    display: flex;
    flex-direction: column;
    align-items: center;
    gap: 8px;
    text-align: center;
    transition: all 0.3s;
}

.summary-card:hover { transform: translateY(-3px); }
.summary-card.red { border-color: rgba(255,82,82,0.3); background: rgba(255,82,82,0.05); }
.summary-card.blue { border-color: rgba(0,212,255,0.3); background: rgba(0,212,255,0.05); }
.summary-card.purple { border-color: rgba(108,99,255,0.3); background: rgba(108,99,255,0.05); }
.summary-card.orange { border-color: rgba(255,179,0,0.3); background: rgba(255,179,0,0.05); }

.summary-card-icon { font-size: 2rem; }
.summary-card-label { font-size: 0.8rem; color: var(--text-secondary); font-weight: 500; }

/* Results Grid */
.results-grid {
    display: grid;
    grid-template-columns: repeat(2, 1fr);
    gap: 24px;
    margin-bottom: 30px;
}

.result-section-card {
    background: var(--bg-card);
    border: 1px solid var(--border);
    border-radius: var(--radius);
    padding: 28px;
    transition: all 0.3s;
}

.result-section-card:hover { border-color: rgba(108,99,255,0.3); }

.result-section-card.abnormal { border-left: 3px solid var(--danger); }
.result-section-card.explanation { border-left: 3px solid var(--secondary); }
.result-section-card.conditions { border-left: 3px solid var(--primary); }
.result-section-card.severity { border-left: 3px solid var(--warning); }
.result-section-card.next-steps { border-left: 3px solid var(--success); }
.result-section-card.full-report { border-left: 3px solid var(--primary); }

.section-card-header {
    display: flex;
    align-items: flex-start;
    gap: 14px;
    margin-bottom: 18px;
    padding-bottom: 16px;
    border-bottom: 1px solid var(--border);
}

.section-icon-wrap {
    width: 42px;
    height: 42px;
    border-radius: 10px;
    display: flex;
    align-items: center;
    justify-content: center;
    font-size: 1.2rem;
    flex-shrink: 0;
}

.red-bg { background: rgba(255,82,82,0.15); }
.blue-bg { background: rgba(0,212,255,0.15); }
.purple-bg { background: rgba(108,99,255,0.15); }
.orange-bg { background: rgba(255,179,0,0.15); }
.green-bg { background: rgba(0,230,118,0.15); }

.section-card-header h3 { font-size: 1rem; font-weight: 700; margin-bottom: 4px; }
.section-card-header p { font-size: 0.78rem; color: var(--text-muted); }

.section-card-body {
    font-size: 0.9rem;
    line-height: 1.8;
    color: var(--text-secondary);
}

/* Disclaimer */
.result-disclaimer {
    display: flex;
    gap: 16px;
    padding: 24px;
    background: rgba(255,179,0,0.06);
    border: 1px solid rgba(255,179,0,0.2);
    border-radius: var(--radius);
    margin-bottom: 30px;
}

.disclaimer-icon { font-size: 2rem; flex-shrink: 0; }
.disclaimer-text strong { font-size: 0.95rem; display: block; margin-bottom: 8px; }
.disclaimer-text p { font-size: 0.85rem; color: var(--text-secondary); line-height: 1.6; }

/* Action Buttons */
.result-actions {
    display: flex;
    gap: 14px;
    align-items: center;
    flex-wrap: wrap;
    margin-bottom: 16px;
}

.share-note {
    text-align: center;
    color: var(--text-muted);
    font-size: 0.82rem;
}

/* ============================================
   PRINT STYLES
   ============================================ */

@media print {
    .navbar, .bg-particles, .result-actions, .share-note { display: none !important; }
    body { background: white; color: black; }
    .result-section-card { border: 1px solid #ccc; break-inside: avoid; }
    .gradient-text { -webkit-text-fill-color: #6C63FF !important; }
}

/* ============================================
   RESPONSIVE
   ============================================ */

@media (max-width: 900px) {
    .hero { flex-direction: column; padding: 60px 24px; gap: 40px; }
    .hero-visual { display: none; }
    .steps-grid, .features-grid { grid-template-columns: 1fr; }
    .analyze-container { grid-template-columns: 1fr; padding: 30px 24px; }
    .results-grid { grid-template-columns: 1fr; }
    .summary-row { grid-template-columns: repeat(2, 1fr); }
    .navbar { padding: 14px 24px; }
    .results-container { padding: 30px 24px; }
}

@media (max-width: 600px) {
    .hero-title { font-size: 2.2rem; }
    .hero-actions { flex-direction: column; align-items: stretch; }
    .stats-bar { flex-wrap: wrap; gap: 12px; }
    .stat-divider { display: none; }
    .summary-row { grid-template-columns: repeat(2, 1fr); }
    .result-actions { flex-direction: column; }
    .btn-primary, .btn-secondary, .btn-ghost { justify-content: center; }
}

/* ============================================
   AUTH PAGES (Login / Register)
   ============================================ */

.auth-page {
    min-height: 100vh;
    display: flex;
    align-items: center;
    justify-content: center;
}

.auth-wrapper {
    position: relative;
    z-index: 1;
    width: 100%;
    max-width: 440px;
    padding: 24px;
}

.auth-card {
    background: rgba(255,255,255,0.04);
    border: 1px solid var(--border);
    border-radius: 24px;
    padding: 40px;
    backdrop-filter: blur(20px);
    box-shadow: var(--shadow-lg);
}

.auth-logo {
    display: flex;
    align-items: center;
    gap: 10px;
    margin-bottom: 28px;
    font-size: 1.3rem;
}

.auth-title {
    font-size: 1.8rem;
    font-weight: 800;
    margin-bottom: 8px;
}

.auth-subtitle {
    color: var(--text-secondary);
    font-size: 0.9rem;
    margin-bottom: 28px;
}

.auth-form {
    display: flex;
    flex-direction: column;
    gap: 18px;
    margin-bottom: 20px;
}

.form-group {
    display: flex;
    flex-direction: column;
    gap: 6px;
}

.form-group label {
    font-size: 0.82rem;
    font-weight: 600;
    color: var(--text-secondary);
    text-transform: uppercase;
    letter-spacing: 0.05em;
}

.form-group input {
    padding: 13px 16px;
    background: rgba(255,255,255,0.06);
    border: 1px solid var(--border);
    border-radius: 12px;
    color: var(--text-primary);
    font-size: 0.95rem;
    font-family: inherit;
    outline: none;
    transition: all 0.3s;
}

.form-group input::placeholder { color: var(--text-muted); }

.form-group input:focus {
    border-color: var(--primary);
    background: rgba(108,99,255,0.08);
    box-shadow: 0 0 0 3px rgba(108,99,255,0.15);
}

.btn-auth {
    width: 100%;
    padding: 15px;
    background: var(--gradient);
    border: none;
    border-radius: 12px;
    color: white;
    font-size: 1rem;
    font-weight: 700;
    cursor: pointer;
    font-family: inherit;
    transition: all 0.3s;
    box-shadow: 0 6px 24px rgba(108,99,255,0.4);
    display: flex;
    align-items: center;
    justify-content: center;
    gap: 8px;
}

.btn-auth:hover {
    transform: translateY(-2px);
    box-shadow: 0 10px 32px rgba(108,99,255,0.6);
}

.auth-switch {
    text-align: center;
    font-size: 0.85rem;
    color: var(--text-muted);
}

.auth-switch a {
    color: var(--primary);
    text-decoration: none;
    font-weight: 600;
    transition: opacity 0.2s;
}

.auth-switch a:hover { opacity: 0.8; }

/* ─── Navbar user badge ─── */
.nav-user {
    display: flex;
    align-items: center;
    gap: 12px;
}

.user-badge {
    display: flex;
    align-items: center;
    gap: 6px;
    padding: 6px 14px;
    background: rgba(108,99,255,0.12);
    border: 1px solid rgba(108,99,255,0.25);
    border-radius: 50px;
    font-size: 0.82rem;
    font-weight: 500;
    color: var(--primary);
}

.logout-link {
    padding: 6px 14px;
    border-radius: 50px;
    color: var(--text-muted);
    text-decoration: none;
    font-size: 0.82rem;
    transition: all 0.3s;
    border: 1px solid transparent;
}

.logout-link:hover {
    color: var(--danger);
    border-color: rgba(255,82,82,0.3);
    background: rgba(255,82,82,0.08);
}

Overwriting static/style.css


In [ ]:
%%writefile static/main.js
// ============================================
//  MEDISCAN AI — main.js
// ============================================

document.addEventListener('DOMContentLoaded', () => {

    // ---- Scroll Animation Observer ----
    const observer = new IntersectionObserver((entries) => {
        entries.forEach(entry => {
            if (entry.isIntersecting) {
                entry.target.classList.add('in-view');
                observer.unobserve(entry.target);
            }
        });
    }, { threshold: 0.1 });

    document.querySelectorAll('.step-card, .feature-card, .report-type-card, .result-section-card').forEach(el => {
        el.style.opacity = '0';
        el.style.transform = 'translateY(30px)';
        el.style.transition = 'opacity 0.6s ease, transform 0.6s ease';
        observer.observe(el);
    });

    // Trigger in-view style
    const style = document.createElement('style');
    style.textContent = '.in-view { opacity: 1 !important; transform: translateY(0) !important; }';
    document.head.appendChild(style);


    // ---- Drag and Drop ----
    const dropZone = document.getElementById('dropZone');
    if (dropZone) {
        dropZone.addEventListener('dragover', (e) => {
            e.preventDefault();
            dropZone.classList.add('drag-over');
        });

        dropZone.addEventListener('dragleave', () => {
            dropZone.classList.remove('drag-over');
        });

        dropZone.addEventListener('drop', (e) => {
            e.preventDefault();
            dropZone.classList.remove('drag-over');
            const file = e.dataTransfer.files[0];
            if (file) {
                const input = document.getElementById('fileInput');
                const dt = new DataTransfer();
                dt.items.add(file);
                input.files = dt.files;
                handleFileSelect(input);
            }
        });
    }

    // ---- Navbar scroll effect ----
    const navbar = document.querySelector('.navbar');
    if (navbar) {
        window.addEventListener('scroll', () => {
            if (window.scrollY > 50) {
                navbar.style.boxShadow = '0 4px 30px rgba(0,0,0,0.4)';
            } else {
                navbar.style.boxShadow = 'none';
            }
        });
    }

    // ---- Animate result cards on result page ----
    document.querySelectorAll('.animate-card').forEach((card, i) => {
        card.style.animationDelay = `${i * 0.1}s`;
        card.style.animationFillMode = 'both';
    });

    // ---- Smooth anchor scroll ----
    document.querySelectorAll('a[href^="#"]').forEach(anchor => {
        anchor.addEventListener('click', function(e) {
            const target = document.querySelector(this.getAttribute('href'));
            if (target) {
                e.preventDefault();
                target.scrollIntoView({ behavior: 'smooth', block: 'start' });
            }
        });
    });

});


// ---- File Select Handler ----
function handleFileSelect(input) {
    const file = input.files[0];
    if (!file) return;

    const preview = document.getElementById('filePreview');
    const nameEl = document.getElementById('fileName');
    const sizeEl = document.getElementById('fileSize');
    const iconEl = document.getElementById('fileIcon');

    if (!preview) return;

    // Set icon based on file type
    const ext = file.name.split('.').pop().toLowerCase();
    const icons = { pdf: '📄', jpg: '🖼️', jpeg: '🖼️', png: '🖼️', txt: '📝' };
    iconEl.textContent = icons[ext] || '📁';

    nameEl.textContent = file.name;
    sizeEl.textContent = formatFileSize(file.size);

    preview.style.display = 'block';

    // Hide drop zone text, show smaller version
    const dropZone = document.getElementById('dropZone');
    if (dropZone) {
        dropZone.style.padding = '24px 40px';
        const content = dropZone.querySelector('.drop-zone-content');
        if (content) content.style.display = 'none';
    }
}


// ---- Remove File ----
function removeFile() {
    const input = document.getElementById('fileInput');
    const preview = document.getElementById('filePreview');
    const dropZone = document.getElementById('dropZone');

    if (input) input.value = '';
    if (preview) preview.style.display = 'none';

    if (dropZone) {
        dropZone.style.padding = '60px 40px';
        const content = dropZone.querySelector('.drop-zone-content');
        if (content) content.style.display = 'block';
    }
}


// ---- Handle Form Submit ----
function handleSubmit(e) {
    const input = document.getElementById('fileInput');
    if (!input || !input.files.length) return;

    const overlay = document.getElementById('loadingOverlay');
    const btnText = document.getElementById('btnText');
    const btnLoader = document.getElementById('btnLoader');

    if (overlay) overlay.style.display = 'flex';
    if (btnText) btnText.style.display = 'none';
    if (btnLoader) btnLoader.style.display = 'flex';

    // Animate loading steps
    const steps = ['step1', 'step2', 'step3', 'step4'];
    const messages = [
        'Uploading your file...',
        'Reading and extracting text...',
        'AI is analyzing your report...',
        'Generating simplified results...'
    ];

    let current = 0;
    const msgEl = document.getElementById('loadingMsg');

    const interval = setInterval(() => {
        current++;
        if (current < steps.length) {
            const stepEl = document.getElementById(steps[current]);
            if (stepEl) stepEl.classList.add('active');
            if (msgEl) msgEl.textContent = messages[current];
        } else {
            clearInterval(interval);
        }
    }, 1800);
}


// ---- Format File Size ----
function formatFileSize(bytes) {
    if (bytes === 0) return '0 B';
    const k = 1024;
    const sizes = ['B', 'KB', 'MB'];
    const i = Math.floor(Math.log(bytes) / Math.log(k));
    return parseFloat((bytes / Math.pow(k, i)).toFixed(1)) + ' ' + sizes[i];
}

Overwriting static/main.js


In [ ]:
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"

^C
^C


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 17431

/bin/bash: line 1: kill: (17431) - No such process


In [ ]:
!nohup python app.py > flask.log 2>&1 &

In [ ]:
# ✅ Start ngrok tunnel
from pyngrok import ngrok, conf
conf.get_default().auth_token = "38goErAyvC538oIc1L90qsGxekz_Cxke6AnLLSwDbqLojxXB"

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ✅ Check Flask logs (useful if error happens)
!sleep 3 && tail -n 20 flask.log

🌍 Public URL: NgrokTunnel: "https://lichenological-variedly-tillie.ngrok-free.dev" -> "http://localhost:8000"
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
Press CTRL+C to quit


In [ ]:
!tail -n 100 flask.log

 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
Press CTRL+C to quit
